In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

In [3]:
def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

SEED = 42
reset_seeds(SEED)

data_path = '/content/drive/MyDrive/AI ML/DACON HANSOL/data/'

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')
train_gpt = pd.read_csv(data_path + 'train_gpt.csv')


# 데이터 정제

In [5]:
train

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
0,TRAIN_000,면진장치가 뭐야?,면진장치에 사용되는 주요 기술은 무엇인가요?,건축구조,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...",면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
1,TRAIN_001,내진설계의 종류 좀 알려줘,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,건축구조,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...","내진설계에는 주로 내진구조, 제진구조, 면진구조의 세 가지 종류가 있습니다. 이들은...","내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조...","내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있..."
2,TRAIN_002,철골구조의 장점이 뭐야?,철골구조의 장점을 알려줘?,건축구조,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건...,철골구조의 장점은 건물의 외벽에 하중이 적게 걸리기 때문에 고층 건물의 건축이 용이...,"철골구조의 장점은 건물의 외벽이 하중이 걸리지 않아 공간 활용이 용이하고, 고층 건...",철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...
3,TRAIN_003,철골철근 콘크리트 구조가 뭐야?,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,건축구조,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근, 그리고 콘크리...",철골철근 콘크리트 구조는 건축물을 지탱하기 위한 구조물에서 일반적으로 사용되는 방식...,"철골철근콘크리트 구조는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건...","철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ..."
4,TRAIN_004,철골구조는 어떤 방식이 있어?,철골구조의 다양한 방식이 무엇인가요?,건축구조,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나...,철골구조는 주로 일반철골구조와 경량철골구조로 나뉘어집니다. 이들은 건축 시스템에 따...,철골구조는 주로 일반철골구조와 경량철골구조로 구분됩니다. 이외에도 최근에는 고층 건...,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...
...,...,...,...,...,...,...,...,...,...
639,TRAIN_639,벽장 부위 결로의 원인이 뭐야?,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,타 마감하자,벽장 부위 결로의 원인은 난방이 이웃한 방과동일한 조건이 되나 그 방에 비해 저온인...,"벽장 부위 결로의 원인은 주로 난방이 자리잡은 방이 내부 온도가 낮은 반면, 외부 ...",벽장 부위 결로가 발생하는 원인은 난방이 잘 이루어지지 않아 해당 공간이 저온인 반...,"벽장 부위 결로가 발생하는 원인은 난방이 인접한 방과 같은 조건을 갖추고 있지만, ...",벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...
640,TRAIN_640,"AD, PD에 면한 벽체 결로의 원인이 뭐야?",벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,타 마감하자,"AD, PD에 면한 벽체 결로의 원인은 외기에 접하는 면 좌, 우측 벽체에는 단열재...","AD, PD에 면한 벽체 결로의 원인은 외기에 접하는 면 좌, 우측 벽체에는 단열재...","AD, PD에 면한 벽체 결로의 주된 원인은 외기에 접하는 면 좌, 우측 벽체에 단...","AD, PD에 면한 벽체 결로의 원인으로는 외기에 접하는 면 좌, 우측 벽체는 일반...","AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각..."
641,TRAIN_641,외벽 모서리 부위에 결로가 발생하는 원인이 뭐야?,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,타 마감하자,외벽모서리 부위에 결로가 발생하는 원인은 높은 온도차 때문입니다. 외벽 모서리의 경...,외벽 모서리 부위에 결로가 발생하는 원인은 중요한 역할을 합니다. 외부에 비치는 세...,외벽 모서리 부위에 결로가 발생하는 원인으로는 높은 온도차가 대표적입니다. 외벽 모...,외벽 모서리 부위에 결로가 발생하는 원인은 실내와 외부의 온도차 때문입니다. 외벽 ...,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...
642,TRAIN_642,창호 결로의 대책은 뭐야?,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,타 마감하자,창호결로의 대책은 제품을 선정할 때 KS에 규정된 프레임을 선정하고 유리의 열관류율...,창호 결로의 대책은 KS에 규정된 프레임을 선정하고 유리의 열관류율 및 결로 발생 ...,"창호 결로를 예방하기 위한 대책은 KS에 규정된 프레임을 선택하고, 열관류율이 적은...","창호 결로의 대책은 여러 가지가 있습니다. 먼저, 창호를 선택할 때 KS에 규정된 ...","창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및..."


In [25]:
category_lst = []
question_lst = []
answer_lst = []
for i in range(1, 3):
    for j in range(1, 6):
        for n in range(train.shape[0]):
            category = train['category'][n]
            question = train[f'질문_{i}'][n]
            answer = train[f'답변_{j}'][n]
            category_lst.append(category)
            question_lst.append(question)
            answer_lst.append(answer)

train_df = pd.DataFrame({'category':category_lst,
                       'question':question_lst,
                       'answer':answer_lst})
train_df

,category,question,answer
0,건축구조,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,건축구조,내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다."
2,건축구조,철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...
3,건축구조,철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ..."
4,건축구조,철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.
...,...,...,...
6435,타 마감하자,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...
6436,타 마감하자,벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,"AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각..."
6437,타 마감하자,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...
6438,타 마감하자,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,"창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및..."


In [9]:
category_lst = []
question_lst = []
answer_lst = []
for i in range(1, 3):
    for j in range(1, 11):
        for n in range(train_gpt.shape[0]):
            category = train_gpt['category'][n]
            question = train_gpt[f'질문_{i}'][n]
            answer = train_gpt[f'답변_{j}'][n]
            category_lst.append(category)
            question_lst.append(question)
            answer_lst.append(answer)

gpt_qna_df = pd.DataFrame({'category':category_lst,
                       'question':question_lst,
                       'answer':answer_lst})
gpt_qna_df

,category,question,answer
0,건축구조,면진장치가 뭐야?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...
1,건축구조,내진설계의 종류 좀 알려줘,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다."
2,건축구조,철골구조의 장점이 뭐야?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...
3,건축구조,철골철근 콘크리트 구조가 뭐야?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ..."
4,건축구조,철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조가 있습니다.
...,...,...,...
12875,타 마감하자,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,벽장 부분에 결로가 생기는 가장 큰 이유는 충분한 환기가 되지 않는 환경과 과도한 ...
12876,타 마감하자,벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,"벽체에 발생하는 결로의 원인 중 하나는, 벽면이 내부보다 외부 공기에 노출되어서 냉..."
12877,타 마감하자,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,외벽 모서리에 결로가 발생하는 주요 이유는 높은 온도차입니다. 외벽 모서리는 열이 ...
12878,타 마감하자,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,KS규격에 따른 프레임 사용과 열전달 및 결로시험 수행은 창호 결로 방지를 위한 필...


In [11]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.8 MB/s eta 0:00:00


In [12]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score

embedding_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

def cosine_score(input_text, generated_text):
    input_embedding = embedding_model.encode(input_text)
    generated_embedding = embedding_model.encode(generated_text)

    cosine_similarity_score = cosine_similarity(input_embedding, generated_embedding)
    cosine_similarity_score = max(cosine_similarity_score, 0)

    return cosine_similarity_score

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
reset_seeds(SEED)

category_list = []
question_list = []
answer_list = []

for i in tqdm(range(train.shape[0])):

    best_similarity = 0
    best_answer = ''

    category = train['category'][i]
    question = train[f'질문_1'][i]

    for a in range(1,6) :
        answer = train[f'답변_{a}'][i]
        similarity = cosine_score(question,answer)

        if similarity > best_similarity :
            best_similarity = similarity
            best_answer = answer

    category_list.append(category)
    question_list.append(question)
    answer_list.append(answer)

qna_df_1 = pd.DataFrame({'category':category_list,
                       'question':question_list,
                       'answer':answer_list})
qna_df_1

  0%|          | 0/644 [00:00<?, ?it/s]

,category,question,answer
0,건축구조,면진장치가 뭐야?,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
1,건축구조,내진설계의 종류 좀 알려줘,"내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있..."
2,건축구조,철골구조의 장점이 뭐야?,철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...
3,건축구조,철골철근 콘크리트 구조가 뭐야?,"철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ..."
4,건축구조,철골구조는 어떤 방식이 있어?,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...
...,...,...,...
639,타 마감하자,벽장 부위 결로의 원인이 뭐야?,벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...
640,타 마감하자,"AD, PD에 면한 벽체 결로의 원인이 뭐야?","AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각..."
641,타 마감하자,외벽 모서리 부위에 결로가 발생하는 원인이 뭐야?,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...
642,타 마감하자,창호 결로의 대책은 뭐야?,"창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및..."


In [16]:
qna_df_1.to_csv(data_path + 'qna_df_1.csv', index=False)

In [17]:
reset_seeds(SEED)

category_list = []
question_list = []
answer_list = []

for i in tqdm(range(train.shape[0])):

    best_similarity = 0
    best_answer = ''

    category = train['category'][i]
    question = train[f'질문_2'][i]

    for a in range(1,6) :
        answer = train[f'답변_{a}'][i]
        similarity = cosine_score(question,answer)

        if similarity > best_similarity :
            best_similarity = similarity
            best_answer = answer

    category_list.append(category)
    question_list.append(question)
    answer_list.append(answer)

qna_df_2 = pd.DataFrame({'category':category_list,
                       'question':question_list,
                       'answer':answer_list})
qna_df_2

  0%|          | 0/644 [00:00<?, ?it/s]

,category,question,answer
0,건축구조,면진장치에 사용되는 주요 기술은 무엇인가요?,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
1,건축구조,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,"내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있..."
2,건축구조,철골구조의 장점을 알려줘?,철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...
3,건축구조,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,"철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ..."
4,건축구조,철골구조의 다양한 방식이 무엇인가요?,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...
...,...,...,...
639,타 마감하자,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...
640,타 마감하자,벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,"AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각..."
641,타 마감하자,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...
642,타 마감하자,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,"창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및..."


In [18]:
qna_df_2.to_csv(data_path + 'qna_df_2.csv', index=False)

In [ ]:
# reset_seeds(SEED)

# category_list = []
# question_list = []
# answer_list = []

# for i in tqdm(range(train.shape[0])):

#     best_similarity = 0
#     best_answer = ''

#     for q in range(1,3) :
#         category = train['category'][i]
#         question = train[f'질문_{q}'][i]

#         for a in range(1,6) :
#             answer = train[f'답변_{a}'][i]
#             similarity = cosine_score(question,answer)

#             if similarity > best_similarity :
#                 best_similarity = similarity
#                 best_answer = answer

#     category_list.append(category)
#     question_list.append(question)
#     answer_list.append(answer)

# qna_df = pd.DataFrame({'category':category_list,
#                        'question':question_list,
#                        'answer':answer_list})
# qna_df



In [21]:
qna_df = pd.read_csv(data_path + 'qna_df_question1-2.csv')
qna_df_1 = pd.read_csv(data_path + 'qna_df_1.csv')
qna_df_2 = pd.read_csv(data_path + 'qna_df_2.csv')

In [29]:
from itertools import combinations, product

reset_seeds(SEED)

index_lst = list(qna_df_1.index)
index_comb_lst = list(combinations(index_lst, 2))
sample_list = random.sample(index_comb_lst, 67120)

aug_category = []
aug_questions = []
aug_answers = []
connect_words = [' 그리고 ', ' 또한, ']
connect_words = list(product(connect_words, repeat=2))

for i, j in sample_list:
    selected_connect = random.choice(connect_words)

    aug_c = qna_df_1['category'][i] + ' + ' + qna_df_1['category'][j]
    aug_q = qna_df_1['question'][i] + selected_connect[0] + qna_df_1['question'][j]
    aug_a = qna_df_1['answer'][i] + selected_connect[1]  + qna_df_1['answer'][j]

    aug_category.append(aug_c)
    aug_questions.append(aug_q)
    aug_answers.append(aug_a)

aug_qna_df_1 = pd.DataFrame({'category':aug_category,
                       'question':aug_questions,
                       'answer':aug_answers})
aug_qna_df_1

,category,question,answer
0,마감하자 + 인테리어,먼지 및 연기로 인해 도배지에 오염이 발생할 수 있어? 그리고 인테리어에서 사용되는...,"도배지 표면에 먼지나 연기가 축적되면 얼룩이 발생할 수 있습니다. 특히, 연기는 도..."
1,기타 + 마감재,"마감재의 하자여부는 어떻게 판단해? 또한, 원목마루의 장점이 뭐야?","마감재의 하자 여부를 판단하는 데 있어서 고려해야 할 사항은, 설계도서와의 일치 여..."
2,건축구조 + 마감재,철골구조의 단점이 뭐야? 그리고 큐블럭의 장점이 뭐야?,철골구조의 주요 단점은 화재에 대한 취약성입니다. 철골은 녹기 쉬운 속성을 가지고 ...
3,시공 + 인테리어,"정배작업이 뭐야? 또한, 다운라이트에 대해 설명해줘","정배작업은 벽지를 자르고, 벽지의 뒷면에 도배용 접착제를 바르는 작업을 말합니다. ..."
4,마감재 + 타 마감하자,쌀풀에 대해 알려줘 그리고 천장 부위 결로의 원인이 뭐야?,"쌀풀은 쌀로 만든 풀을 지칭하는 용어로, 친환경적이고 밀풀에 비해 약 1.5배의 높..."
...,...,...,...
67115,마감하자 + 인테리어,"도배지에 울음이 생겼어 또한, 2023년의 최신 인테리어 디자인 트렌드가 무엇인가요?",도배지 울음이란 도배지가 평평하지 않고 국소적으로 볼록하게 튀어나오는 현상을 말합니...
67116,마감재 + 시공,"그라스울의 단점이 뭐야? 또한, 액체방수공사가 뭐야?",그라스울은 내부마감을 꼼꼼히 하지 않을 경우 가루가 누출될 수 있어서 관리가 조심스...
67117,마감재 + 마감하자,포세린 타일이 뭐야? 그리고 부적절한 접착제를 사용하면 도배지꼬임이 발생할 수 있어?,포세린 타일은 점토를 기본 재료로 사용하여 만든 무광의 타일을 말합니다. 이러한 타...
67118,마감하자 + 타 마감하자,"구조적 결함으로 인해 석고수정이 발생할 수 있어? 또한, 결로 판정 기준이 어떻게 돼?",건물의 구조적 결함은 석고수정의 발생을 유발할 수 있습니다. 구조적 결함은 건축물의...


In [28]:
from itertools import combinations, product

reset_seeds(SEED)

index_lst = list(qna_df_2.index)
index_comb_lst = list(combinations(index_lst, 2))
sample_list = random.sample(index_comb_lst, 67120)

aug_category = []
aug_questions = []
aug_answers = []
connect_words = [' 그리고 ', ' 또한, ']
connect_words = list(product(connect_words, repeat=2))

for i, j in sample_list:
    selected_connect = random.choice(connect_words)

    aug_c = qna_df_2['category'][i] + ' + ' + qna_df_2['category'][j]
    aug_q = qna_df_2['question'][i] + selected_connect[0] + qna_df_2['question'][j]
    aug_a = qna_df_2['answer'][i] + selected_connect[1]  + qna_df_2['answer'][j]

    aug_category.append(aug_c)
    aug_questions.append(aug_q)
    aug_answers.append(aug_a)

aug_qna_df_2 = pd.DataFrame({'category':aug_category,
                       'question':aug_questions,
                       'answer':aug_answers})
aug_qna_df_2

,category,question,answer
0,마감하자 + 인테리어,도배지에 먼지 및 연기로 인한 오염이 발생할 경우 어떻게 처리해야 하나요? 그리고 ...,"도배지 표면에 먼지나 연기가 축적되면 얼룩이 발생할 수 있습니다. 특히, 연기는 도..."
1,기타 + 마감재,"마감재의 하자를 어떻게 확인할 수 있을까요? 또한, 원목마루를 사용하는 장점은 무엇...","마감재의 하자 여부를 판단하는 데 있어서 고려해야 할 사항은, 설계도서와의 일치 여..."
2,건축구조 + 마감재,철골구조의 장점을 알려줘 그리고 큐블럭을 사용하는 이유는 무엇인가요?,철골구조의 주요 단점은 화재에 대한 취약성입니다. 철골은 녹기 쉬운 속성을 가지고 ...
3,시공 + 인테리어,"정배작업은 건설 현장에서 사용되는 작업 방식으로, 어떤 상황에서 정배작업을 사용하는...","정배작업은 벽지를 자르고, 벽지의 뒷면에 도배용 접착제를 바르는 작업을 말합니다. ..."
4,마감재 + 타 마감하자,쌀풀은 무엇이며 어떤 용도로 사용되나요? 그리고 천장 부위 결로가 발생하는 가장 일...,"쌀풀은 쌀로 만든 풀을 지칭하는 용어로, 친환경적이고 밀풀에 비해 약 1.5배의 높..."
...,...,...,...
67115,마감하자 + 인테리어,"도배지에 울음이 생기는 이유가 뭔가요? 또한, 2023년의 인기 있는 인테리어 디자...",도배지 울음이란 도배지가 평평하지 않고 국소적으로 볼록하게 튀어나오는 현상을 말합니...
67116,마감재 + 시공,"그라스울의 단점은 무엇인가요? 또한, 액체방수공사는 어떤 공사인가요?",그라스울은 내부마감을 꼼꼼히 하지 않을 경우 가루가 누출될 수 있어서 관리가 조심스...
67117,마감재 + 마감하자,포세린 타일의 장점과 단점에 대해서 알고 싶어요. 그리고 접착제를 잘못 사용하면 도...,포세린 타일은 점토를 기본 재료로 사용하여 만든 무광의 타일을 말합니다. 이러한 타...
67118,마감하자 + 타 마감하자,"구조적 결함으로 인해 석고수정이 발생할 가능성이 있는가요? 또한, 결로가 발생하는 ...",건물의 구조적 결함은 석고수정의 발생을 유발할 수 있습니다. 구조적 결함은 건축물의...


In [41]:
qna_df.shape, qna_df_1.shape, qna_df_2.shape,  aug_qna_df_1.shape, aug_qna_df_2.shape, train_df.shape, gpt_qna_df.shape

((644, 3), (644, 3), (644, 3), (67120, 3), (67120, 3), (6440, 3), (12880, 3))

In [39]:
644*3

1932

In [145]:
train_ft = pd.concat([aug_qna_df_1,gpt_qna_df,qna_df, qna_df_1, qna_df_2]).reset_index(drop=True)

train_ft

,category,question,answer
0,마감하자 + 인테리어,먼지 및 연기로 인해 도배지에 오염이 발생할 수 있어? 그리고 인테리어에서 사용되는...,"도배지 표면에 먼지나 연기가 축적되면 얼룩이 발생할 수 있습니다. 특히, 연기는 도..."
1,기타 + 마감재,"마감재의 하자여부는 어떻게 판단해? 또한, 원목마루의 장점이 뭐야?","마감재의 하자 여부를 판단하는 데 있어서 고려해야 할 사항은, 설계도서와의 일치 여..."
2,건축구조 + 마감재,철골구조의 단점이 뭐야? 그리고 큐블럭의 장점이 뭐야?,철골구조의 주요 단점은 화재에 대한 취약성입니다. 철골은 녹기 쉬운 속성을 가지고 ...
3,시공 + 인테리어,"정배작업이 뭐야? 또한, 다운라이트에 대해 설명해줘","정배작업은 벽지를 자르고, 벽지의 뒷면에 도배용 접착제를 바르는 작업을 말합니다. ..."
4,마감재 + 타 마감하자,쌀풀에 대해 알려줘 그리고 천장 부위 결로의 원인이 뭐야?,"쌀풀은 쌀로 만든 풀을 지칭하는 용어로, 친환경적이고 밀풀에 비해 약 1.5배의 높..."
...,...,...,...
81927,타 마감하자,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...
81928,타 마감하자,벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,"AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각..."
81929,타 마감하자,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...
81930,타 마감하자,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,"창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및..."


In [50]:
1932/80000 # 2% 정도만 중복 데이터 허용

0.02415

In [51]:
# mask = train_ft.duplicated(keep='first')

# train_ft = train_ft[~mask]
# train_ft

# 데이터 정제

In [146]:
import re

re_question_list = []
re_answer_list = []

for i in tqdm(range(len(train_ft))):
    question = train_ft.iloc[i]['question']
    answer = train_ft.iloc[i]['answer']

    pat = re.compile("[^a-zA-Z가-힣ㄱ-ㅎ0-9 .,\-%!?\"]")

    question = pat.sub('', question)
    answer = pat.sub('', answer)

    re_question_list.append(question)
    re_answer_list.append(answer)

train_ft['question'] = re_question_list
train_ft['answer'] = re_answer_list

train_ft

100%|██████████| 81932/81932 [00:07<00:00, 11430.66it/s]


,category,question,answer
0,마감하자 + 인테리어,먼지 및 연기로 인해 도배지에 오염이 발생할 수 있어? 그리고 인테리어에서 사용되는...,"도배지 표면에 먼지나 연기가 축적되면 얼룩이 발생할 수 있습니다. 특히, 연기는 도..."
1,기타 + 마감재,"마감재의 하자여부는 어떻게 판단해? 또한, 원목마루의 장점이 뭐야?","마감재의 하자 여부를 판단하는 데 있어서 고려해야 할 사항은, 설계도서와의 일치 여..."
2,건축구조 + 마감재,철골구조의 단점이 뭐야? 그리고 큐블럭의 장점이 뭐야?,철골구조의 주요 단점은 화재에 대한 취약성입니다. 철골은 녹기 쉬운 속성을 가지고 ...
3,시공 + 인테리어,"정배작업이 뭐야? 또한, 다운라이트에 대해 설명해줘","정배작업은 벽지를 자르고, 벽지의 뒷면에 도배용 접착제를 바르는 작업을 말합니다. ..."
4,마감재 + 타 마감하자,쌀풀에 대해 알려줘 그리고 천장 부위 결로의 원인이 뭐야?,"쌀풀은 쌀로 만든 풀을 지칭하는 용어로, 친환경적이고 밀풀에 비해 약 1.5배의 높..."
...,...,...,...
81927,타 마감하자,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...
81928,타 마감하자,벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,"AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각..."
81929,타 마감하자,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...
81930,타 마감하자,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,"창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및..."


In [147]:
# 문장에 관한 문장
mask = train_ft['answer'].str.contains('이 문장')

train_ft = train_ft[~mask].reset_index(drop=True)
train_ft

,category,question,answer
0,마감하자 + 인테리어,먼지 및 연기로 인해 도배지에 오염이 발생할 수 있어? 그리고 인테리어에서 사용되는...,"도배지 표면에 먼지나 연기가 축적되면 얼룩이 발생할 수 있습니다. 특히, 연기는 도..."
1,기타 + 마감재,"마감재의 하자여부는 어떻게 판단해? 또한, 원목마루의 장점이 뭐야?","마감재의 하자 여부를 판단하는 데 있어서 고려해야 할 사항은, 설계도서와의 일치 여..."
2,건축구조 + 마감재,철골구조의 단점이 뭐야? 그리고 큐블럭의 장점이 뭐야?,철골구조의 주요 단점은 화재에 대한 취약성입니다. 철골은 녹기 쉬운 속성을 가지고 ...
3,시공 + 인테리어,"정배작업이 뭐야? 또한, 다운라이트에 대해 설명해줘","정배작업은 벽지를 자르고, 벽지의 뒷면에 도배용 접착제를 바르는 작업을 말합니다. ..."
4,마감재 + 타 마감하자,쌀풀에 대해 알려줘 그리고 천장 부위 결로의 원인이 뭐야?,"쌀풀은 쌀로 만든 풀을 지칭하는 용어로, 친환경적이고 밀풀에 비해 약 1.5배의 높..."
...,...,...,...
81903,타 마감하자,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...
81904,타 마감하자,벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,"AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각..."
81905,타 마감하자,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...
81906,타 마감하자,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,"창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및..."


In [148]:
train_ft

,category,question,answer
0,마감하자 + 인테리어,먼지 및 연기로 인해 도배지에 오염이 발생할 수 있어? 그리고 인테리어에서 사용되는...,"도배지 표면에 먼지나 연기가 축적되면 얼룩이 발생할 수 있습니다. 특히, 연기는 도..."
1,기타 + 마감재,"마감재의 하자여부는 어떻게 판단해? 또한, 원목마루의 장점이 뭐야?","마감재의 하자 여부를 판단하는 데 있어서 고려해야 할 사항은, 설계도서와의 일치 여..."
2,건축구조 + 마감재,철골구조의 단점이 뭐야? 그리고 큐블럭의 장점이 뭐야?,철골구조의 주요 단점은 화재에 대한 취약성입니다. 철골은 녹기 쉬운 속성을 가지고 ...
3,시공 + 인테리어,"정배작업이 뭐야? 또한, 다운라이트에 대해 설명해줘","정배작업은 벽지를 자르고, 벽지의 뒷면에 도배용 접착제를 바르는 작업을 말합니다. ..."
4,마감재 + 타 마감하자,쌀풀에 대해 알려줘 그리고 천장 부위 결로의 원인이 뭐야?,"쌀풀은 쌀로 만든 풀을 지칭하는 용어로, 친환경적이고 밀풀에 비해 약 1.5배의 높..."
...,...,...,...
81903,타 마감하자,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...
81904,타 마감하자,벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,"AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각..."
81905,타 마감하자,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...
81906,타 마감하자,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,"창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및..."


In [149]:
# 최소 중복단어

idx_lst = []
for i in tqdm(range(len(train_ft))):
    question = train_ft.iloc[i]['question']
    answer = train_ft.iloc[i]['answer']

    p = re.compile(r'\b(\w+)\s+\1\b')
    if p.search(answer):
        print(f'index{i} 중복 내용 : {p.search(answer)}')
        print(answer)
        print(' ')
        idx_lst.append(i)


 84%|████████▍ | 68645/81908 [00:10<00:01, 8564.77it/s]

index67488 중복 내용 : <re.Match object; span=(47, 56), match='발생하는 발생하는'>
도배지의 오염얼룩은 국소적으로 점, 선, 면의 형태를 이루고 있습니다. 얼룩 하자가 발생하는 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인  높은 습도 환경에서 도배지의 표면이 젖으면 다양한 물질이 흡수되기 쉽습니다. 이러한 물질은 표면에 얼룩 형태로 나타날 수 있습니다. 책임소재   건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법  제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 먼지 및 연기로 인한 오염 원인  도배지 표면에 먼지나 연기담배, 조리과정에서 발생하는 연기 등가 축적되면 얼룩이 발생할 수 있습니다. 책임소재  건물의 소유자나 거주자가 환경 관리의 책임이 있습니다. 해결 방법  필요한 경우 얼룩 제거제나 청소 용품을 사용하여 얼룩을 처리해야 합니다. 얼룩이 사라지지 않으면 부분 및 전체 도배 작업하는 것을 고려해야 합니다. 3. 화학물질에 의한 오염 원인  도배지가 화학물질에 노출되면 오염이 발생합니다. 책임소재  건물의 소유자나 거주자가 책임이 있습니다. 해결 방법  화학물질의 종류를 파악한 후 알맞은 방법으로 오염을 제거해야 하며 만약 오염이 제거되지 않는다면 벽지를 교체해야 합니다.
 
index67506 중복 내용 : <re.Match object; span=(38, 47), match='발생하는 발생하는'>
벽이나 천장에 잘못 뚫린 구멍은 "타공하자"라고 합니다. 타공하자가 발생하는 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 부실 시공 원인  적절하지 않은 도구로 작업하거나 타공 부위의 크기를 정확히 측정하지 않고 작업하면 타공불량이 발생할 수 있습니다. 책임소재  시공자 해결 방법  타공 부위의 크기를 정확히 측정하고 올바른 도구로 재작업해야 합니다.
 
i

 90%|████████▉ | 73378/81908 [00:10<00:00, 9305.48it/s]

index72027 중복 내용 : <re.Match object; span=(121, 126), match='다음 다음'>
도배풀이 바닥이나 몰딩, 벽지에 붙은 경우, 부드러운 걸레로 부드럽게 닦아내야 하고, 제거되지 않는 경우 도배풀 제거제를 사용하여 완전히 깨끗하게 제거할 수 있습니다. 도배풀을 완전히 제거한 후 청소 잔해물을 제거한 다음 다음 단계로 진행해야 합니다.
 
index73916 중복 내용 : <re.Match object; span=(47, 56), match='발생하는 발생하는'>
도배지의 오염얼룩은 국소적으로 점, 선, 면의 형태를 이루고 있습니다. 얼룩 하자가 발생하는 발생하는 원인 - 책임소재 - 해결방법에 대해 설명드리겠습니다. 1. 공간 내 높은 습도 원인  높은 습도 환경에서 도배지의 표면이 젖으면 다양한 물질이 흡수되기 쉽습니다. 이러한 물질은 표면에 얼룩 형태로 나타날 수 있습니다. 책임소재   건물의 소유자나 거주자가 습기 관리의 책임이 있습니다. 해결 방법  제습기 가동, 환기를 통해 실내 적정 습도를 유지하고 전문가의 도움을 받아 보수작업하는 것을 추천합니다. 2. 먼지 및 연기로 인한 오염 원인  도배지 표면에 먼지나 연기담배, 조리과정에서 발생하는 연기 등가 축적되면 얼룩이 발생할 수 있습니다. 책임소재  건물의 소유자나 거주자가 환경 관리의 책임이 있습니다. 해결 방법  필요한 경우 얼룩 제거제나 청소 용품을 사용하여 얼룩을 처리해야 합니다. 얼룩이 사라지지 않으면 부분 및 전체 도배 작업하는 것을 고려해야 합니다. 3. 화학물질에 의한 오염 원인  도배지가 화학물질에 노출되면 오염이 발생합니다. 책임소재  건물의 소유자나 거주자가 책임이 있습니다. 해결 방법  화학물질의 종류를 파악한 후 알맞은 방법으로 오염을 제거해야 하며 만약 오염이 제거되지 않는다면 벽지를 교체해야 합니다.
 
index73934 중복 내용 : <re.Match object; span=(38, 47), match='발생하는 발생하는'>
벽이나

 93%|█████████▎| 76320/81908 [00:11<00:00, 9640.46it/s]

index74806 중복 내용 : <re.Match object; span=(124, 133), match='선인장과 선인장과'>
실내 식물을 가꾸려면 먼저 공기 정화식물, 간편한 관리식물, 그리고 조명과 습도에 맞는 식물을 선택해야 합니다. 예를 들어 습도가 높은 욕실이나 주방에는 아레카야스 같은 공기 정화식물이 적합하며, 햇빛이 잘 드는 창가에는 선인장과 선인장과 같은 간편한 관리식물을 선택할 수 있습니다. 그 외에도 관리 난이도에 따라 식물을 선택해야 합니다.
 


 97%|█████████▋| 79286/81908 [00:11<00:00, 9801.15it/s]

index78455 중복 내용 : <re.Match object; span=(121, 126), match='다음 다음'>
도배풀이 바닥이나 몰딩, 벽지에 붙은 경우, 부드러운 걸레로 부드럽게 닦아내야 하고, 제거되지 않는 경우 도배풀 제거제를 사용하여 완전히 깨끗하게 제거할 수 있습니다. 도배풀을 완전히 제거한 후 청소 잔해물을 제거한 다음 다음 단계로 진행해야 합니다.
 


100%|██████████| 81908/81908 [00:11<00:00, 6891.38it/s]


In [150]:
train_ft = train_ft.drop(idx_lst)
train_ft = train_ft.reset_index(drop=True)

train_ft


,category,question,answer
0,마감하자 + 인테리어,먼지 및 연기로 인해 도배지에 오염이 발생할 수 있어? 그리고 인테리어에서 사용되는...,"도배지 표면에 먼지나 연기가 축적되면 얼룩이 발생할 수 있습니다. 특히, 연기는 도..."
1,기타 + 마감재,"마감재의 하자여부는 어떻게 판단해? 또한, 원목마루의 장점이 뭐야?","마감재의 하자 여부를 판단하는 데 있어서 고려해야 할 사항은, 설계도서와의 일치 여..."
2,건축구조 + 마감재,철골구조의 단점이 뭐야? 그리고 큐블럭의 장점이 뭐야?,철골구조의 주요 단점은 화재에 대한 취약성입니다. 철골은 녹기 쉬운 속성을 가지고 ...
3,시공 + 인테리어,"정배작업이 뭐야? 또한, 다운라이트에 대해 설명해줘","정배작업은 벽지를 자르고, 벽지의 뒷면에 도배용 접착제를 바르는 작업을 말합니다. ..."
4,마감재 + 타 마감하자,쌀풀에 대해 알려줘 그리고 천장 부위 결로의 원인이 뭐야?,"쌀풀은 쌀로 만든 풀을 지칭하는 용어로, 친환경적이고 밀풀에 비해 약 1.5배의 높..."
...,...,...,...
81895,타 마감하자,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...
81896,타 마감하자,벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,"AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각..."
81897,타 마감하자,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...
81898,타 마감하자,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,"창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및..."


In [151]:
train_ft.to_csv('train_ft_81900_0304.csv', index=False)

In [152]:
train_ft.to_csv(f'{data_path}train_ft_81900_0304.csv', index=False)

In [153]:
# train_ft.to_excel(f'{data_path}train_ft_81900_0304.xlsx', index=False)
# train_ft.to_excel('train_ft_81900_0304.xlsx', index=False,encoding='utf-8')

/usr/local/lib/python3.10/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


# 학습시키기

In [5]:
train_ft = pd.read_csv(f'{data_path}train_ft_81900_0304.csv')
train_ft

,category,question,answer
0,마감하자 + 인테리어,먼지 및 연기로 인해 도배지에 오염이 발생할 수 있어? 그리고 인테리어에서 사용되는...,"도배지 표면에 먼지나 연기가 축적되면 얼룩이 발생할 수 있습니다. 특히, 연기는 도..."
1,기타 + 마감재,"마감재의 하자여부는 어떻게 판단해? 또한, 원목마루의 장점이 뭐야?","마감재의 하자 여부를 판단하는 데 있어서 고려해야 할 사항은, 설계도서와의 일치 여..."
2,건축구조 + 마감재,철골구조의 단점이 뭐야? 그리고 큐블럭의 장점이 뭐야?,철골구조의 주요 단점은 화재에 대한 취약성입니다. 철골은 녹기 쉬운 속성을 가지고 ...
3,시공 + 인테리어,"정배작업이 뭐야? 또한, 다운라이트에 대해 설명해줘","정배작업은 벽지를 자르고, 벽지의 뒷면에 도배용 접착제를 바르는 작업을 말합니다. ..."
4,마감재 + 타 마감하자,쌀풀에 대해 알려줘 그리고 천장 부위 결로의 원인이 뭐야?,"쌀풀은 쌀로 만든 풀을 지칭하는 용어로, 친환경적이고 밀풀에 비해 약 1.5배의 높..."
...,...,...,...
81895,타 마감하자,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...
81896,타 마감하자,벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,"AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각..."
81897,타 마감하자,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...
81898,타 마감하자,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,"창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및..."


In [6]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.6 MB/s eta 0:00:00


In [7]:
from transformers import GPT2LMHeadModel, AutoTokenizer

model_name = 'skt/kogpt2-base-v2'

model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

In [8]:
qna_data = []
for _, row in tqdm(train_ft.iterrows()):
    input_text = "<q>" + row['question'] + "</s><a>" + row['answer'] + "</s>"
    qna_data.append(input_text)

qna_df = pd.DataFrame({'qna':qna_data})
qna_df

0it [00:00, ?it/s]

,qna
0,<q>먼지 및 연기로 인해 도배지에 오염이 발생할 수 있어? 그리고 인테리어에서 사...
1,"<q>마감재의 하자여부는 어떻게 판단해? 또한, 원목마루의 장점이 뭐야?</s><a..."
2,<q>철골구조의 단점이 뭐야? 그리고 큐블럭의 장점이 뭐야?</s><a>철골구조의 ...
3,"<q>정배작업이 뭐야? 또한, 다운라이트에 대해 설명해줘</s><a>정배작업은 벽지..."
4,<q>쌀풀에 대해 알려줘 그리고 천장 부위 결로의 원인이 뭐야?</s><a>쌀풀은 ...
...,...
81895,<q>벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?</s><a>벽장 부위 결로...
81896,"<q>벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?</s><a>AD, ..."
81897,<q>외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?</s><a...
81898,<q>창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?</s><a>창호 ...


In [13]:
qna_df.to_csv(f'{data_path}qna_df_81900_0304.csv', index=False)

In [9]:
class DobaeDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.question = df['qna'].tolist()

    def __len__(self):
        return len(self.question)

    def __getitem__(self, idx):
        return self.question[idx]

In [10]:
def collate_fn(batch):
    x = tokenizer(batch, return_tensors='pt', padding=True)
    return {'x': x}

In [11]:
dt = DobaeDataset(qna_df)
dl = torch.utils.data.DataLoader(dt,batch_size=2,collate_fn=collate_fn)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[ 9724,   455,   405,  7514,  8263,  9140,  9081,  9926, 10407,  9095,
           7609,  9844, 13464,  8146, 17756,  9025,  9341,   406,  9394, 50059,
           9023, 13138,  9059,  9902, 18250,  9863,  9199,  9019,  7055,  8084,
            406,     1,  9724,   439,   405,  7235,  7609,  8263, 16284, 10199,
          15806, 43245, 16828, 12577, 25071,  8146, 17756,  9025, 32987, 12518,
           9081,  9446,  9095,  7609,  9844, 13464,  8137, 21578,  9025, 12072,
          27867,  9300, 10115,  9676, 13993, 25051,  9368, 13266,  9095,  7609,
           9750, 13464,  8137, 17031, 15154,  9199,  9677, 16691, 10783, 50059,
           9023,  9584, 13138,  9059,  9902, 28808,  9544,  9059, 10059, 15659,
           9059, 10059, 11895,  7767, 37723, 10050, 37723, 20223,  9059, 10059,
           9230,  8664, 46211,  9225, 32987, 14477,  9059, 10935, 20369, 34024,
           9384,  9626,  9887,  9418, 27711, 13163, 17160,  9059,  9902, 20092,
           9161, 4447

In [12]:
def train_loop(dataloader,model,loss_fn,optimizer,device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):
        x = batch["x"].to(device)
        pred = model(**x).logits
        n_class = pred.shape[-1]
        pred = pred[:, :-1]
        pred = pred.reshape(-1, n_class)

        tgt = x["input_ids"][:, 1:]
        tgt = tgt.flatten()

        mask = tgt != tokenizer.pad_token_id
        tgt = tgt[mask]
        pred = pred[mask]
        loss = loss_fn(pred,tgt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

In [16]:
batch_size = 4
lr = 3e-5
loss_fn = torch.nn.CrossEntropyLoss()
epochs = 10

In [ ]:
reset_seeds(SEED)

model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
train_dt = DobaeDataset(qna_df)
train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

for i in tqdm(range(epochs)):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)

    print(f'{i+1}번째 epoch: {train_loss:.5f}')

    # if i >= 3:
    model.save_pretrained(f'{data_path}model/kogpt2_89000_0304_{i+1}epoch_{train_loss:.5f}loss')

    torch.cuda.empty_cache()

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/20475 [00:00<?, ?it/s]

1번째 epoch: 0.34763


  0%|          | 0/20475 [00:00<?, ?it/s]

2번째 epoch: 0.18782


  0%|          | 0/20475 [00:00<?, ?it/s]

3번째 epoch: 0.15429


  0%|          | 0/20475 [00:00<?, ?it/s]

4번째 epoch: 0.13556


  0%|          | 0/20475 [00:00<?, ?it/s]